In [1]:
import json 
import numpy as pd
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:
with open ('../input/intents/intents.json') as file:
    data=json.load(file)

In [3]:
training_sentences=[]
training_labels=[]
labels=[]
responses=[]

for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

In [4]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [5]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [6]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

2022-02-16 19:36:48.101250: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 136       
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
_________________________________________________________________


In [8]:
epochs = 550
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/550


2022-02-16 19:36:48.357003: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


2/2 [==============================] - 1s 5ms/step - loss: 2.0781 - accuracy: 0.1515
Epoch 2/550
2/2 [==============================] - 0s 5ms/step - loss: 2.0766 - accuracy: 0.1212
Epoch 3/550
2/2 [==============================] - 0s 4ms/step - loss: 2.0759 - accuracy: 0.2121
Epoch 4/550
2/2 [==============================] - 0s 4ms/step - loss: 2.0755 - accuracy: 0.2121
Epoch 5/550
2/2 [==============================] - 0s 4ms/step - loss: 2.0752 - accuracy: 0.2424
Epoch 6/550
2/2 [==============================] - 0s 4ms/step - loss: 2.0748 - accuracy: 0.2424
Epoch 7/550
2/2 [==============================] - 0s 3ms/step - loss: 2.0743 - accuracy: 0.2424
Epoch 8/550
2/2 [==============================] - 0s 3ms/step - loss: 2.0741 - accuracy: 0.1515
Epoch 9/550
2/2 [==============================] - 0s 3ms/step - loss: 2.0740 - accuracy: 0.1212
Epoch 10/550
2/2 [==============================] - 0s 3ms/step - loss: 2.0738 - accuracy: 0.0909
Epoch 11/550
2/2 [=======================

In [9]:
# saving model
model.save("chat_model")

import pickle

# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

# saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

2022-02-16 19:36:55.602232: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
